# Anomaly Detection using K-Means and Spectral Clustering

## First: Importing the necessary libraries

In [3]:
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

## Second: Importing the dataset and preprocessing it

- We first load the data from the CSVs.
- We then proceed with preprocessing the data as follows:
    1. Split the labels from the data (last column) and place them in separate dataframes.
    2. Encode the categorical data using into one-hot vectors using the OneHotEncoder class.
    3. Scale the features using the StandardScaler class.

In [31]:
# Loading the data as indicated in the assignment
# kddcup.data.corrected is the training data, corrected is the testing data
training_data = pd.read_csv( 'archive/kddcup.data.corrected', header=None )
testing_data = pd.read_csv( 'archive/corrected', header=None )

# Separating the labels (last column) from the features in the training and testing data
X_train = training_data.iloc[:, :-1]
y_train = training_data.iloc[:, -1]

X_test = testing_data.iloc[:, :-1]
y_test = testing_data.iloc[:, -1]

# Concatenating the training and testing data vertically to perform OneHotEncoding on all the categorical features
X = pd.concat( [X_train, X_test] )

# Encoding the features into one-hot vectors using OneHotEncoder
# ColumnTransformer is used to apply the OneHotEncoder to the second, third and fourth columns (categorical features)
# The remainder is set to 'passthrough' to keep the other columns unchanged (numerical features)
col_trans = ColumnTransformer( transformers=[('encoder', OneHotEncoder(), [1, 2, 3])], remainder='passthrough' )
X = col_trans.fit_transform(X)

# Separating the training and testing data again after OneHotEncoding
X_train = X[:len(X_train)]
X_test = X[len(X_train):]

# Feature Scaling since some features have a much higher range than others
scaler = StandardScaler()
X_train = pd.DataFrame( scaler.fit_transform( X_train ) )
X_test = pd.DataFrame( scaler.transform( X_test ) )

In [32]:
# Randomly picking 10000 samples from the training data to speed up the training process (for testing purposes only)
X_train = X_train.sample(10000, random_state=0)

In [33]:
try:
    print('Number of unique labels in the training data: ', len(y_train.unique()))
    print('-' * 30)
    print('Number of unique labels in the testing data: ', len(y_test.unique()))
except NameError:
    print('\033[91m' + 'NameError: y_train or y_test is not defined' + '\033[0m')

Number of unique labels in the training data:  23
------------------------------
Number of unique labels in the testing data:  38


## Third: Applying K-Means and Normalized Cut

### Supplementary Functions

In [26]:
# This function will be used to calculate the euclidean distance between two data points
def euclidean_distance( row, centroid, data ):
    distance = 0.0
    for column in data.columns:
        distance += (row[column] - centroid[column])**2
    return np.sqrt(distance)


# This function will be used to calculate the mean of the data points in a cluster.
def calculate_mean( data ):
    mean = pd.DataFrame(columns=data.columns)
    for column in data.columns:
        mean[column] = [data[column].mean()]
    return mean
    

# This function returns the ANSI code for bold text
def bold( text, reset=True ):
    if reset:
        return '\033[1m' + text + '\033[0m'
    return '\033[1m' + text

# This function returns the ANSI code for underlined text
def underline( text, reset=True ):
    if reset:
        return '\033[4m' + text + '\033[0m'
    return '\033[4m' + text

### K-Means Clustering Algorithm

#### Implementation

In [27]:
def kmeans_clustering( k, data, max_iterations:int=None, print_updates=False, initial_centroids=None ):
    
    # Initially, selecting k random data points as centroids
    # We will use the current time as the seed to make sure that we get different centroids each time we run the algorithm
    np.random.seed( int(time.time()) )

    if initial_centroids is None:
        centroids = data.sample(k)
    else:
        centroids = initial_centroids.copy()
        
    old_centroids = None

    # If the user doesn't specify the maximum number of iterations, we will set it to infinity (Loop until convergence)
    if max_iterations is None:
        max_iterations = np.inf

    itr = 1
    while( itr <= max_iterations ):

        # If the centroids do not change, we will stop the algorithm
        if centroids.equals( old_centroids ):
            break

        # Storing the old centroids to check if they change in the next iteration
        old_centroids = centroids.copy()

        if print_updates is True: print( underline(bold('Iteration #' + str(itr))) )

        # Creating a dictionary to store the clusters and their data points
        # 'clusters' will store the data points and 'cluster_indices' will store their indices
        clusters, cluster_indices = {}, {}
        for i in range(k):
            clusters[i] = []
            cluster_indices[i] = []

        # Iterating through each data point and assigning it to the closest cluster
        for index, row in data.iterrows():

            min_distance, closest_cluster_index = np.inf, -1
            
            # Iterating through each centroid to find the closest one
            for i in range(k):

                # Using our euclidean_distance function to calculate the distance as it handles both numerical and categorical data
                current_distance = euclidean_distance( row, centroids.iloc[i], data )

                # Check if the data point is closer to the ith centroid
                if current_distance < min_distance:
                    min_distance = current_distance
                    closest_cluster_index = i
            
            # Assigning the data point to the cluster with the closest centroid
            clusters[closest_cluster_index].append( row )
            cluster_indices[closest_cluster_index].append( index )

        # Updating the centroids.
        # We will use our calculate_mean function to calculate the mean of the data points in the cluster
        # because it handles both numerical and categorical data
        for i in range(k):

            # If the cluster is empty, we will not update the centroid
            if len(clusters[i]) == 0:
                continue
            else:
                centroids.iloc[i] = calculate_mean( pd.DataFrame(clusters[i]) )

        # Printing the cluster sizes in tabular form to conserve vertical space
        if print_updates is True:
            print( 'Cluster sizes:' )
            print(pd.DataFrame( [len(clusters[i]) for i in range(k)] ).T)

        if print_updates is True: print('-' * 50) # Just to print a line to separate the iterations

        itr += 1

    return centroids, clusters, cluster_indices

#### Execution

In [28]:
# This function will be used to calculate the purity of the clusters
# Purity is the percentage of data points in a cluster that belong to the same class
def calculate_purity( clusters, labels, print_report=False ):
    purity = 0.0
    purities = []
    for i in range( len(clusters) ):
        cluster = clusters[i]

        # If the cluster is empty, we will skip it
        if len(cluster) == 0: continue

        # Converting the cluster to a dataframe so that we can use the value_counts() function
        cluster = pd.DataFrame(cluster)
        cluster['label'] = labels[cluster.index]

        # We will use the value_counts() function to count the number of data points in each class
        # and then we will divide it by the total number of data points in the cluster
        purities.append( cluster['label'].value_counts()[0] / len(cluster) )

    # Normalizing the purity by dividing it by the number of clusters
    average_purity = sum(purities) / len(clusters)

    if print_report is True:
        for i in range(len(purities)):
            print('Cluster ', i+1, ' purity: ', purities[i])
        print('-'*50)
        print('Average Purity: ', average_purity)
        print('-'*50)
    
    return average_purity, purities


# This function prints a report of the clusters produced by the k-means algorithm
def analyze_clusters( clusters, cluster_indices, labels ):

    # Printing the number of data points in each cluster
    for i in range(len(cluster_indices)):
        print('Cluster ', i+1, ' contains ', len(cluster_indices[i]), ' data points')
    print( '-' * 50 )

    # Calculating the purity of the clusters and printing the report
    calculate_purity( clusters, labels, print_report=True )

    # Printing the number of unique labels in each cluster
    for i in range(len(cluster_indices)):
        print( 'Cluster #' + str(i+1) + ' labels:\n' )
        print(labels[cluster_indices[i]].value_counts())
        print('-' * 50)

In [41]:
try:
    centroids2, clusters2, cluster_indices2 = kmeans_clustering( k=10, data=X_train, print_updates=True )
except KeyboardInterrupt:
    print('\033[91m' + 'Process interrupted by user' + '\033[0m')

Iteration #1


In [40]:
try:
    analyze_clusters( clusters2, cluster_indices2, y_train )
except NameError:
    print('\033[91m' + 'NameError: clusters2 or cluster_indices2 is not defined' + '\033[0m')

Cluster  1  contains  5708  data points
Cluster  2  contains  4292  data points
--------------------------------------------------
Cluster  1  purity:  0.9973721093202523
Cluster  2  purity:  0.5263280521901211
--------------------------------------------------
Average Purity:  0.7618500807551867
--------------------------------------------------
Cluster #1 labels:

smurf.      5693
normal.       14
ipsweep.       1
Name: 41, dtype: int64
--------------------------------------------------
Cluster #2 labels:

neptune.        2259
normal.         1950
satan.            29
portsweep.        22
ipsweep.          18
back.              6
nmap.              4
teardrop.          2
warezclient.       2
Name: 41, dtype: int64
--------------------------------------------------


In [ ]:
try:
    centroids23, clusters23, cluster_indices23 = kmeans_clustering( k=23, data=X_test, max_iterations=5, print_updates=True )
except KeyboardInterrupt:
    print('\033[91m' + 'Process interrupted by user' + '\033[0m')

In [ ]:
try:
    analyze_clusters( clusters23, cluster_indices23, y_test )
except NameError:
    print('\033[91m' + 'Error: No clusters found' + '\033[0m')

### Normalized Cut Algorithm

#### Implementation (Not vectorized)

In [ ]:
# This function measures the normalized cut of a graph, given the weight matrix and the clusters resulting from the graph cut
# weight_matrix expects a numpy array, and clusters expects a list of lists where each list contains the indices of the nodes in the cluster
def measure_cut( weight_matrix, clusters ):
    
    # Calculating the cut
    # Here, we will iterate through each pair of clusters and calculate the sum of the weights of the edges between them
    # We add the cut between the ith and jth clusters to the total cut measure which will then be used to calculate the normalized cut
    total_cut_measure = 0
    for i in range( len(clusters) ):
        for j in range( len(clusters) ):

            # We don't want to calculate the cut between a cluster and itself
            # So, we will skip the iteration if i == j
            if i == j: continue

            # Calculating the cut between the ith and jth clusters
            cut += np.sum( weight_matrix[clusters[i], :][:, clusters[j]] )

    # Calculating the volume
    # Here, we will iterate through each cluster and calculate the sum of the weights of the edges inside the cluster
    total_volume = 0
    for i in range( len(clusters) ):
        total_volume += np.sum( weight_matrix[clusters[i], :][:, clusters[i]] )

    # Calculating the normalized cut
    normalized_cut = total_cut_measure / total_volume

    return normalized_cut, total_cut_measure